# Задача классификации семейств лягушек по издаваемым звукам

Цели работы: 

1. Потренироваться в использовании стекинга для создания моделей классификации.
2. Построить модель, определяющую лягушек семейства 'Dendrobatidae'.

Основа ипользуемых данных - информация о звуках, издаваемых лягушками, и характеристики этих звуков.

Результаты:
1. Построена модель с использованием методики стекинга.
2. Итоговое значение метрики F1 - 0.99

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.metrics import f1_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression

In [2]:
#Посмотрим на данные
frogs = pd.read_csv('Frogs_MFCCs.csv')
frogs.head()

,MFCCs_ 1,MFCCs_ 2,MFCCs_ 3,MFCCs_ 4,MFCCs_ 5,MFCCs_ 6,MFCCs_ 7,MFCCs_ 8,MFCCs_ 9,MFCCs_10,...,MFCCs_17,MFCCs_18,MFCCs_19,MFCCs_20,MFCCs_21,MFCCs_22,Family,Genus,Species,RecordID
0,1.0,0.152936,-0.105586,0.200722,0.317201,0.260764,0.100945,-0.150063,-0.171128,0.124676,...,-0.108351,-0.077623,-0.009568,0.057684,0.118680,0.014038,Leptodactylidae,Adenomera,AdenomeraAndre,1
1,1.0,0.171534,-0.098975,0.268425,0.338672,0.268353,0.060835,-0.222475,-0.207693,0.170883,...,-0.090974,-0.056510,-0.035303,0.020140,0.082263,0.029056,Leptodactylidae,Adenomera,AdenomeraAndre,1
2,1.0,0.152317,-0.082973,0.287128,0.276014,0.189867,0.008714,-0.242234,-0.219153,0.232538,...,-0.050691,-0.023590,-0.066722,-0.025083,0.099108,0.077162,Leptodactylidae,Adenomera,AdenomeraAndre,1
3,1.0,0.224392,0.118985,0.329432,0.372088,0.361005,0.015501,-0.194347,-0.098181,0.270375,...,-0.136009,-0.177037,-0.130498,-0.054766,-0.018691,0.023954,Leptodactylidae,Adenomera,AdenomeraAndre,1
4,1.0,0.087817,-0.068345,0.306967,0.330923,0.249144,0.006884,-0.265423,-0.172700,0.266434,...,-0.048885,-0.053074,-0.088550,-0.031346,0.108610,0.079244,Leptodactylidae,Adenomera,AdenomeraAndre,1


In [3]:
#Смотрим количество классов:
frogs['Family'].value_counts()

#Классов всего 4. А нам надо прогнозировать один из них. 

Leptodactylidae    4420
Hylidae            2165
Dendrobatidae       542
Bufonidae            68
Name: Family, dtype: int64

In [4]:
#Делим классы на 'Dendrobatidae' = 1 и все остальные = 0
frogs['Family'] = frogs['Family'].apply(lambda x: 1 if x=='Dendrobatidae' else 0)
frogs.sample(5)

,MFCCs_ 1,MFCCs_ 2,MFCCs_ 3,MFCCs_ 4,MFCCs_ 5,MFCCs_ 6,MFCCs_ 7,MFCCs_ 8,MFCCs_ 9,MFCCs_10,...,MFCCs_17,MFCCs_18,MFCCs_19,MFCCs_20,MFCCs_21,MFCCs_22,Family,Genus,Species,RecordID
5099,1.000000,0.708728,0.661881,0.323970,-0.005024,0.105432,-0.073410,0.110639,0.194059,-0.265758,...,-0.109493,-0.030310,0.070900,0.133870,0.010425,-0.180163,0,Hypsiboas,HypsiboasCinerascens,36
6106,1.000000,0.152550,0.350817,0.313831,0.165520,0.193691,0.124401,-0.130999,-0.039587,0.080869,...,0.121362,0.033514,-0.004452,-0.050959,-0.073919,0.022301,0,Hypsiboas,HypsiboasCordobae,41
5341,0.965492,0.691062,1.000000,0.411179,-0.301598,0.223071,0.267353,0.049172,0.324450,0.085916,...,-0.143597,0.016998,0.146978,0.091630,0.001310,-0.129489,0,Hypsiboas,HypsiboasCinerascens,37
3245,1.000000,0.217794,0.083049,0.410470,0.196130,0.075045,-0.066397,0.007376,0.203176,0.080309,...,0.215655,0.101541,-0.117708,-0.193933,0.003375,0.227470,0,Adenomera,AdenomeraHylaedactylus,21
4962,1.000000,-0.198880,-0.126623,0.563248,0.353166,0.155062,0.048250,-0.111703,-0.044987,0.311319,...,-0.216799,-0.043211,0.086997,0.195963,0.099531,-0.151352,0,Dendropsophus,HylaMinuta,34


In [5]:
#Сверяем количество строк в классах
frogs['Family'].value_counts()


0    6653
1     542
Name: Family, dtype: int64

In [6]:
#Делим данные на признаки и таргет
X = frogs.drop(['Family','Genus','Species','RecordID'], axis=1)
y = frogs['Family']


In [7]:
#Делим данные на тренировочные и тестовые
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=31)


In [8]:
#Строим модель случайного леса
rf = RandomForestClassifier(n_estimators=10, random_state=42)
rf.fit(X_train, y_train)
y_test_pred = rf.predict(X_test)
f1_score(y_test, y_test_pred)


0.9726775956284154

Теперь попробуем улучшить качество нашего предсказания за счёт использования стекинга.

В качестве базовых моделей выберем следующие:

- случайный лес с десятью деревьями, random_state = 31;
- KNN, количество соседей = 11;
- наивный байесовский классификатор с параметрами по умолчанию, в качестве метода возьмите GaussianNB().

В качестве метамодели выберем логистическую регрессию.

Обучим модели и сделаем прогноз целевой метки для тестового набора данных.

Рассчитаем F1-меру для тестового набора данных.

In [9]:
#Создаем список кортежей вида: (наименование модели, модель)
estimators = [
    ('rf', RandomForestClassifier(random_state=31)),
    ('knn',  KNeighborsClassifier(n_neighbors=11)),
    ('nb', GaussianNB())
]

In [10]:
#Создаем объект класса стекинг
reg = StackingClassifier(
    estimators=estimators,
    final_estimator=LogisticRegression()
)
 
#Обучаем модель
reg.fit(X_train, y_train)


StackingClassifier(estimators=[('rf', RandomForestClassifier(random_state=31)),
                               ('knn', KNeighborsClassifier(n_neighbors=11)),
                               ('nb', GaussianNB())],
                   final_estimator=LogisticRegression())

In [11]:
#Делаем прогноз
y_pred_stack = reg.predict(X_test)
print(f'Качество прогноза по F1-score для стекинга {round(f1_score(y_test, y_pred_stack),2)}')


Качество прогноза по F1-score для стекинга 0.99


In [12]:
#Смотрим получившиеся метапризнаки - результат работы трёх алгоритмов. С ними работает логистическая регрессия.
meta_data = reg.transform(X_train)
#Создаем DataFrame
meta_df = pd.DataFrame(
    meta_data, #содержимое таблицы
    columns=['meta_feature1', 'meta_feature2', 'meta_feature3'] #название столбцов
)
meta_df.head()


,meta_feature1,meta_feature2,meta_feature3
0,0.00,0.0,9.629904e-01
1,0.00,0.0,1.663535e-08
2,0.87,1.0,9.999992e-01
3,0.00,0.0,9.052580e-16
4,0.00,0.0,2.378892e-25
